In [1]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 10.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install webdriver-manager


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

# 크롬 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 브라우저를 안 띄우고 실행 (테스트용)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 예시: 멜론 락/메탈 차트 접속
url = "https://www.melon.com/chart/month/index.htm?classCd=GN1000"
driver.get(url)

time.sleep(3)  # 페이지 로딩 대기

# 여기서부터 크롤링 코드 작성!

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

# 크롬 드라이버 세팅
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저 안 띄움
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 월별 URL 생성 함수
def get_month_url(year, month):
    return f"https://www.melon.com/chart/month/index.htm?classCd=GN1000&ymd={year}{month:02d}01"

# 곡 순위 데이터 저장용
all_data = {}

# 2025년 1~7월 루프
for month in range(1, 8):
    url = get_month_url(2025, month)
    driver.get(url)
    time.sleep(3)

    # HTML 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    rows = soup.select('tbody tr')

    for rank, row in enumerate(rows, 1):
        title_tag = row.select_one('div.ellipsis.rank01 a')
        artist_tag = row.select_one('div.ellipsis.rank02 a')
        
        if title_tag and artist_tag:
            title = title_tag.text.strip()
            artist = artist_tag.text.strip()
            key = (title, artist)

            if key not in all_data:
                all_data[key] = [0]*12  # 12개월 초기화

            all_data[key][month - 1] = rank  # 해당 월의 순위 저장

# 종료
driver.quit()

# 데이터프레임 생성
df = pd.DataFrame([
    [title, artist, *ranks, sum(ranks)/sum(1 for r in ranks if r > 0)]
    for (title, artist), ranks in all_data.items()
], columns=["Title", "Artist", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec", "AvgRank"])

# 0은 이탈이므로 평균에서 제외되며, 순위 낮을수록 인기 많음
df = df.sort_values(by="AvgRank")
df.reset_index(drop=True, inplace=True)
df.head(10)


,Title,Artist,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,AvgRank
0,Sugar,Maroon 5,1,1,1,1,1,1,1,0,0,0,0,0,1.0
1,Viva La Vida,Coldplay,2,2,2,2,2,2,2,0,0,0,0,0,2.0
2,Believer,Imagine Dragons,3,3,3,3,3,3,3,0,0,0,0,0,3.0
3,High Hopes,Panic! At The Disco,4,4,4,4,4,4,4,0,0,0,0,0,4.0
4,Lost Stars,Adam Levine,5,5,5,5,5,5,5,0,0,0,0,0,5.0
5,Payphone (Feat. Wiz Khalifa),Maroon 5,6,6,6,6,6,6,6,0,0,0,0,0,6.0
6,Circles,Post Malone,7,7,7,7,7,7,7,0,0,0,0,0,7.0
7,Photograph,Ed Sheeran,8,8,8,8,8,8,8,0,0,0,0,0,8.0
8,I'm Yours,Jason Mraz,9,9,9,9,9,9,9,0,0,0,0,0,9.0
9,Beggin',Måneskin,10,10,10,10,10,10,10,0,0,0,0,0,10.0


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

# 1. 셀레니움 드라이버 설정
options = Options()
options.add_argument("--headless")  # 브라우저를 띄우지 않음
driver = webdriver.Chrome(options=options)

# 2. 월별 수집 대상
months = ['202501', '202502', '202503', '202504', '202505', '202506', '202507']  # 2025년 1월 ~ 7월

# 3. 데이터 저장용 딕셔너리
ranking_data = {}

# 4. 각 월별 데이터 수집
for month in months:
    url = f'https://www.melon.com/chart/month/index.htm?classCd=GN1000&ymd={month}'
    driver.get(url)
    time.sleep(3)  # 페이지 로드 대기

    # iframe 존재 시 전환 필요 (멜론은 iframe은 없음)
    
    try:
        # 차트 데이터 추출
        rows = driver.find_elements(By.CSS_SELECTOR, 'tbody tr')
        for rank, row in enumerate(rows, start=1):
            try:
                title = row.find_element(By.CSS_SELECTOR, 'div.ellipsis.rank01 a').text
                artist = row.find_element(By.CSS_SELECTOR, 'div.ellipsis.rank02 a').text
                key = (title, artist)

                if key not in ranking_data:
                    ranking_data[key] = {m: 0 for m in months}

                ranking_data[key][month] = rank
            except:
                continue
    except Exception as e:
        print(f'{month} 데이터 수집 중 오류 발생: {e}')
        continue

# 5. 데이터프레임 변환
df = pd.DataFrame([
    {'곡명': title, '아티스트': artist, **ranks}
    for (title, artist), ranks in ranking_data.items()
])

# 6. 결과 확인
print(df.head())


             곡명                 아티스트  202501  202502  202503  202504  202505  \
0         Sugar             Maroon 5       1       1       1       1       1   
1  Viva La Vida             Coldplay       2       2       2       2       2   
2      Believer      Imagine Dragons       3       3       3       3       3   
3    High Hopes  Panic! At The Disco       4       4       4       4       4   
4    Lost Stars          Adam Levine       5       5       5       5       5   

   202506  202507  
0       1       1  
1       2       2  
2       3       3  
3       4       4  
4       5       5  


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# 셀레니움 드라이버 설정
options = Options()
# options.add_argument('--headless')  # 화면 안 띄우고 실행하려면 주석 해제
driver = webdriver.Chrome(options=options)

# 수집 대상 월 리스트 (2025년 1월~7월)
months = ['2025.01', '2025.02', '2025.03', '2025.04', '2025.05', '2025.06', '2025.07']
ranking_data = {}

# 사이트 접속
driver.get("https://www.melon.com/chart/month/index.htm?classCd=GN1000")
wait = WebDriverWait(driver, 10)

for month in months:
    # 1. 월 선택 드롭다운 열기
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".arrow_d"))).click()
    time.sleep(0.5)

    # 2. 드롭다운에서 해당 월 클릭
    month_xpath = f"//a[text()='{month}']"
    wait.until(EC.element_to_be_clickable((By.XPATH, month_xpath))).click()
    time.sleep(3)  # 페이지 로딩 대기

    # 3. 곡 리스트 수집
    rows = driver.find_elements(By.CSS_SELECTOR, 'tbody tr')
    for rank, row in enumerate(rows, start=1):
        try:
            title = row.find_element(By.CSS_SELECTOR, 'div.ellipsis.rank01 a').text
            artist = row.find_element(By.CSS_SELECTOR, 'div.ellipsis.rank02 a').text
            key = (title, artist)

            if key not in ranking_data:
                ranking_data[key] = {m: 0 for m in months}

            ranking_data[key][month] = rank
        except:
            continue

# 4. 데이터프레임으로 변환
df = pd.DataFrame([
    {'곡명': title, '아티스트': artist, **ranks}
    for (title, artist), ranks in ranking_data.items()
])

# 5. CSV 저장
df.to_csv("melon_rock_2025.csv", index=False, encoding="utf-8-sig")

driver.quit()
print(df.head())


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000103323570 cxxbridge1$str$ptr + 2731064
1   chromedriver                        0x000000010331b468 cxxbridge1$str$ptr + 2698032
2   chromedriver                        0x0000000102e6a3f8 cxxbridge1$string$len + 90664
3   chromedriver                        0x0000000102eb171c cxxbridge1$string$len + 382284
4   chromedriver                        0x0000000102ef2b1c cxxbridge1$string$len + 649548
5   chromedriver                        0x0000000102ea5a0c cxxbridge1$string$len + 333884
6   chromedriver                        0x00000001032e65f4 cxxbridge1$str$ptr + 2481340
7   chromedriver                        0x00000001032e985c cxxbridge1$str$ptr + 2494244
8   chromedriver                        0x00000001032c7248 cxxbridge1$str$ptr + 2353424
9   chromedriver                        0x00000001032ea118 cxxbridge1$str$ptr + 2496480
10  chromedriver                        0x00000001032b82f8 cxxbridge1$str

In [7]:
# 맨 앞에 '!'를 붙여서 터미널 명령어 실행
!pip install selenium webdriver-manager playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 5.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
128.4 MiB [                    ] 0% 0.0s128.4 MiB [                    ] 0% 63.2s128.4 MiB [                    ] 0% 51.6s128.4 MiB [                    ] 0% 54.5s128.4 MiB [                    ] 0% 52.5s128.4 MiB [                    ] 0% 50.5s128.4 MiB [                    ] 0% 47.4s128.4 MiB [                    ] 0% 42.8s128.4 MiB [                    ] 0% 40.0s128.4 MiB [                    ] 0% 38.9s128.4 MiB [                    ] 0% 42.9s128.4 MiB [                    ] 0% 43.9s128.4 MiB [                    ] 0% 40.5s128.4 MiB [                    ] 0% 39.8s128.4 MiB [                    ] 0% 39.5s128.4 MiB [                    ] 0% 38.8s128.4 MiB [                    ] 0% 39.0s128.4 MiB [                    ] 0% 38.2s128.4 MiB [                    ] 0% 37.5s128.4 MiB [            

In [8]:
import asyncio
from playwright.async_api import async_playwright

async def scrape_multiple_pages():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        
        await page.goto("http://quotes.toscrape.com")
        print("✅ 1페이지로 이동했습니다.")

        # 총 2개의 페이지만 크롤링
        for i in range(2):
            print(f"\n scraping --- {i + 1}페이지 크롤링 시작 ---")

            # 현재 페이지에 있는 모든 명언(div.quote)을 가져옴
            quotes = await page.locator("div.quote").all()
            
            # 각 명언에서 텍스트와 작성자를 추출
            for quote in quotes:
                text = await quote.locator(".text").inner_text()
                author = await quote.locator(".author").inner_text()
                print(f" - {author}: {text[:30]}...") # 너무 길어서 30자만 출력

            # 다음 페이지로 넘어가는 'Next' 버튼을 찾음
            next_button = page.locator("li.next a")
            
            # 다음 페이지가 있다면(마지막 페이지가 아니라면)
            if await next_button.is_visible():
                print("✅ Next 버튼을 클릭하여 다음 페이지로 이동합니다.")
                await next_button.click()
                # 페이지가 완전히 로드될 때까지 기다림
                await page.wait_for_load_state("networkidle") 
            else:
                print("❗️ 다음 페이지가 없습니다. 크롤링을 종료합니다.")
                break # 반복문 종료

        await asyncio.sleep(3)
        await browser.close()
        print("\n--- 모든 작업 완료 ---")

# 위에서 정의한 비동기 함수를 실행
await scrape_multiple_pages()

✅ 1페이지로 이동했습니다.

 scraping --- 1페이지 크롤링 시작 ---
 - Albert Einstein: “The world as we have created ...
 - J.K. Rowling: “It is our choices, Harry, tha...
 - Albert Einstein: “There are only two ways to li...
 - Jane Austen: “The person, be it gentleman o...
 - Marilyn Monroe: “Imperfection is beauty, madne...
 - Albert Einstein: “Try not to become a man of su...
 - André Gide: “It is better to be hated for ...
 - Thomas A. Edison: “I have not failed. I've just ...
 - Eleanor Roosevelt: “A woman is like a tea bag; yo...
 - Steve Martin: “A day without sunshine is lik...
✅ Next 버튼을 클릭하여 다음 페이지로 이동합니다.

 scraping --- 2페이지 크롤링 시작 ---
 - Marilyn Monroe: “This life is what you make it...
 - J.K. Rowling: “It takes a great deal of brav...
 - Albert Einstein: “If you can't explain it to a ...
 - Bob Marley: “You may not be her first, her...
 - Dr. Seuss: “I like nonsense, it wakes up ...
 - Douglas Adams: “I may not have gone where I i...
 - Elie Wiesel: “The opposite of love is not h...
 - Frie

In [9]:
import asyncio
from playwright.async_api import async_playwright
import random # 일정한 간격이 아닌, 랜덤한 간격을 주기 위해 import

async def scrape_naver_news():
    print("🚀 네이버 뉴스 크롤링을 시작합니다...")
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # 1. 네이버 뉴스 IT/과학 섹션으로 이동
        url = "https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105"
        await page.goto(url)
        print(f"✅ '{url}' 페이지로 이동했습니다.")

        # 2. 기사 목록이 로딩될 때까지 잠시 대기
        #    'sh_list_item_wrap' 이라는 클래스를 가진 요소가 나타날 때까지 기다립니다.
        await page.wait_for_selector("div.sh_list_item_wrap")
        print("✅ 기사 목록 로딩을 확인했습니다.")

        # 3. 페이지에 있는 기사 컨테이너를 모두 가져옴
        articles = await page.locator("div.sh_list_item_wrap").all()
        print(f"✅ 총 {len(articles)}개의 기사를 발견했습니다. 수집을 시작합니다.")
        
        # 수집한 데이터를 저장할 리스트
        news_list = []

        # 4. 각 기사를 순회하며 정보 추출
        for article in articles:
            # 5. 기사 제목(<a> 태그)과 언론사(<div> 태그)를 찾음
            headline_element = article.locator("a.sh_text_headline")
            press_element = article.locator("div.sh_text_press")

            # 6. 텍스트와 링크(href) 정보 추출
            title = await headline_element.inner_text()
            press = await press_element.inner_text()
            link = await headline_element.get_attribute('href')
            
            # 7. 수집한 정보를 딕셔너리 형태로 저장하고 리스트에 추가
            news_item = {
                "언론사": press.strip(),
                "제목": title.strip(),
                "링크": link
            }
            news_list.append(news_item)
            
            print(f"  - 수집 완료: [{press.strip()}] {title.strip()}")

            # 8. 🤖 '착한 크롤러'의 핵심: 서버에 부담을 주지 않기 위해 랜덤한 시간(1~2초)동안 대기
            await asyncio.sleep(random.uniform(1, 2))

        # 9. 브라우저 닫기
        await browser.close()
        
        print("\n🎉 크롤링이 성공적으로 완료되었습니다!")
        # 수집된 전체 결과 확인
        # for item in news_list:
        #    print(item)
            
# 위에서 정의한 비동기 함수를 실행
await scrape_naver_news()

🚀 네이버 뉴스 크롤링을 시작합니다...
✅ 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105' 페이지로 이동했습니다.


TimeoutError: Page.wait_for_selector: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("div.sh_list_item_wrap") to be visible


In [10]:
import asyncio
from playwright.async_api import async_playwright
import random

async def scrape_naver_news_updated():
    print("🚀 네이버 뉴스 크롤링을 시작합니다...")
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        url = "https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105"
        await page.goto(url)
        print(f"✅ '{url}' 페이지로 이동했습니다.")

        # 1. (수정) 새로운 기사 목록 선택자로 변경
        list_selector = "div.sa_item_inner"
        await page.wait_for_selector(list_selector)
        print("✅ 기사 목록 로딩을 확인했습니다.")

        articles = await page.locator(list_selector).all()
        print(f"✅ 총 {len(articles)}개의 기사를 발견했습니다. 수집을 시작합니다.")
        
        news_list = []

        for article in articles:
            # 2. (수정) 기사 제목과 언론사의 새로운 선택자로 변경
            headline_element = article.locator("a.sa_text_title")
            press_element = article.locator("div.sa_item_press")

            title = await headline_element.inner_text()
            press = await press_element.inner_text()
            link = await headline_element.get_attribute('href')
            
            news_item = {
                "언론사": press.strip(),
                "제목": title.strip(),
                "링크": link
            }
            news_list.append(news_item)
            
            print(f"  - 수집 완료: [{press.strip()}] {title.strip()}")
            await asyncio.sleep(random.uniform(1, 2))

        await browser.close()
        
        print("\n🎉 크롤링이 성공적으로 완료되었습니다!")
        # 수집된 전체 결과 확인
        # for item in news_list:
        #    print(item)
            
await scrape_naver_news_updated()

🚀 네이버 뉴스 크롤링을 시작합니다...
✅ 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105' 페이지로 이동했습니다.
✅ 기사 목록 로딩을 확인했습니다.
✅ 총 44개의 기사를 발견했습니다. 수집을 시작합니다.


TimeoutError: Locator.inner_text: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("div.sa_item_inner").first.locator("div.sa_item_press")


In [11]:
import asyncio
from playwright.async_api import async_playwright
import random

async def scrape_naver_news_final():
    print("🚀 네이버 뉴스 크롤링을 시작합니다...")
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        url = "https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105"
        await page.goto(url)
        print(f"✅ '{url}' 페이지로 이동했습니다.")

        list_selector = "div.sa_item_inner"
        await page.wait_for_selector(list_selector)
        print("✅ 기사 목록 로딩을 확인했습니다.")

        articles = await page.locator(list_selector).all()
        print(f"✅ 총 {len(articles)}개의 기사를 발견했습니다. 수집을 시작합니다.")
        
        news_list = []

        for article in articles:
            # 🤖 여기서부터 try...except 구문으로 감싸줍니다.
            try:
                # 기존의 데이터 추출 코드를 try 블록 안에 넣습니다.
                headline_element = article.locator("a.sa_text_title")
                press_element = article.locator("div.sa_item_press")

                # 이 부분들에서 오류가 발생할 수 있습니다.
                title = await headline_element.inner_text()
                press = await press_element.inner_text()
                link = await headline_element.get_attribute('href')
                
                news_item = { "언론사": press.strip(), "제목": title.strip(), "링크": link }
                news_list.append(news_item)
                
                print(f"  - 수집 완료: [{press.strip()}] {title.strip()}")

            except Exception as e:
                # try 블록에서 오류가 발생하면 이 코드가 실행됩니다.
                # 프로그램을 멈추는 대신, 메시지를 출력하고 다음 기사로 넘어갑니다.
                print(f"  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.")
                continue # 다음 루프로 바로 넘어감

            # '착한 크롤러'를 위한 대기 시간
            await asyncio.sleep(random.uniform(1, 2))

        await browser.close()
        
        print("\n🎉 크롤링이 성공적으로 완료되었습니다!")
        print(f"총 {len(news_list)}개의 유효한 기사를 수집했습니다.")
        # 수집된 전체 결과 확인
        # for item in news_list:
        #    print(item)
            
await scrape_naver_news_final()

🚀 네이버 뉴스 크롤링을 시작합니다...
✅ 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=105' 페이지로 이동했습니다.
✅ 기사 목록 로딩을 확인했습니다.
✅ 총 44개의 기사를 발견했습니다. 수집을 시작합니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  - ❗️ 구조가 다른 항목(광고, 포토 등)을 발견하여 건너뜁니다.
  -

In [12]:
import asyncio
from playwright.async_api import async_playwright
import random

async def scrape_dc_gallery_10pages():
    print("🚀 '특이점이 온다' 갤러리 정보/뉴스 탭 크롤링을 시작합니다...")
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # 1. 베이스 URL 설정 (페이지 번호는 {}로 비워둠)
        base_url = "https://gall.dcinside.com/mgallery/board/lists/?id=singlebungle&search_head=10&page={}"
        
        # 수집한 모든 게시글을 저장할 리스트
        all_posts = []

        # 2. 1페이지부터 10페이지까지 반복
        for page_num in range(1, 11):
            # 현재 페이지 번호에 맞춰 전체 URL 생성
            url = base_url.format(page_num)
            await page.goto(url)
            print(f"\n✅ {page_num}페이지로 이동했습니다. ({url})")
            
            await page.wait_for_selector("tbody")

            # 3. 해당 페이지의 모든 일반 게시글(tr.ub-content)을 가져옴
            posts_in_page = await page.locator("tr.ub-content").all()
            
            if not posts_in_page:
                print("❗️ 해당 페이지에 게시글이 없습니다. 다음 페이지로 넘어갑니다.")
                continue

            print(f"✅ {len(posts_in_page)}개의 게시글을 발견했습니다. 수집을 시작합니다.")

            for post in posts_in_page:
                try:
                    # 4. 각 게시글에서 정보 추출
                    post_num = await post.locator("td.gall_num").inner_text()
                    title = await post.locator("td.gall_tit a").first.inner_text()
                    writer = await post.locator("td.gall_writer").get_attribute('data-nick')

                    post_item = {"페이지": page_num, "번호": post_num, "제목": title, "글쓴이": writer}
                    all_posts.append(post_item)
                    # print(f"  - 수집: [{post_num}] {title}") # 너무 많이 출력되므로 주석 처리

                except Exception as e:
                    print(f"  - ❗️ 항목 처리 중 오류 발생. 건너뜁니다.")
                    continue
            
            # 5. 서버에 부담을 주지 않기 위한 랜덤 대기
            await asyncio.sleep(random.uniform(1, 2.5))

        await browser.close()
        
        print("\n🎉 크롤링이 성공적으로 완료되었습니다!")
        print(f"총 {len(all_posts)}개의 게시글을 수집했습니다.")
        
        # 수집된 결과 중 처음 5개와 마지막 5개만 출력해서 확인
        print("\n--- 수집 결과 (일부) ---")
        for item in all_posts[:5]:
           print(item)
        print("...")
        for item in all_posts[-5:]:
           print(item)
            
await scrape_dc_gallery_10pages()

🚀 '특이점이 온다' 갤러리 정보/뉴스 탭 크롤링을 시작합니다...


TimeoutError: Page.goto: Timeout 30000ms exceeded.
Call log:
  - navigating to "https://gall.dcinside.com/mgallery/board/lists/?id=singlebungle&search_head=10&page=1", waiting until "load"


In [13]:
import asyncio
from playwright.async_api import async_playwright
import random

async def scrape_dc_gallery_new_url():
    print("🚀 '특이점이 온다' 갤러리(thesingularity) 크롤링을 시작합니다...")
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        # 🤖 요청하신 새로운 URL 주소로 이 부분만 수정했습니다.
        base_url = "https://gall.dcinside.com/mgallery/board/lists/?id=thesingularity&sort_type=N&search_head=10&page={}"
        
        all_posts = []

        for page_num in range(1, 11):
            url = base_url.format(page_num)
            
            try:
                # 페이지 이동 (안정성을 위해 domcontentloaded 옵션 사용)
                await page.goto(url, wait_until="domcontentloaded", timeout=60000)
                print(f"\n✅ {page_num}페이지로 이동했습니다.")
                
                await page.wait_for_selector("tbody")

                posts_in_page = await page.locator("tr.ub-content").all()
                
                if not posts_in_page:
                    print("❗️ 해당 페이지에 게시글이 없습니다. 크롤링을 중단합니다.")
                    break

                print(f"✅ {len(posts_in_page)}개의 게시글을 발견했습니다.")

                for post in posts_in_page:
                    try:
                        post_num = await post.locator("td.gall_num").inner_text()
                        title = await post.locator("td.gall_tit a").first.inner_text()
                        writer = await post.locator("td.gall_writer").get_attribute('data-nick')
                        post_item = {"페이지": page_num, "번호": post_num, "제목": title, "글쓴이": writer}
                        all_posts.append(post_item)
                    except Exception as e:
                        print(f"  - ❗️ 항목 처리 중 오류 발생. 건너뜁니다.")
                        continue
                
                await asyncio.sleep(random.uniform(1, 2.5))
            
            except Exception as e:
                print(f"❗️ {page_num}페이지 로딩 중 심각한 오류 발생: {e}")
                continue

        await browser.close()
        
        print("\n🎉 크롤링이 성공적으로 완료되었습니다!")
        print(f"총 {len(all_posts)}개의 게시글을 수집했습니다.")
        
        if all_posts:
            print("\n--- 수집 결과 (일부) ---")
            for item in all_posts[:5]:
               print(item)
            print("...")
            for item in all_posts[-5:]:
               print(item)
            
await scrape_dc_gallery_new_url()

🚀 '특이점이 온다' 갤러리(thesingularity) 크롤링을 시작합니다...

✅ 1페이지로 이동했습니다.
✅ 47개의 게시글을 발견했습니다.

✅ 2페이지로 이동했습니다.
✅ 51개의 게시글을 발견했습니다.

✅ 3페이지로 이동했습니다.
✅ 51개의 게시글을 발견했습니다.

✅ 4페이지로 이동했습니다.
✅ 51개의 게시글을 발견했습니다.

✅ 5페이지로 이동했습니다.
✅ 51개의 게시글을 발견했습니다.

✅ 6페이지로 이동했습니다.
✅ 51개의 게시글을 발견했습니다.

✅ 7페이지로 이동했습니다.
✅ 51개의 게시글을 발견했습니다.

✅ 8페이지로 이동했습니다.
✅ 51개의 게시글을 발견했습니다.

✅ 9페이지로 이동했습니다.
✅ 51개의 게시글을 발견했습니다.

✅ 10페이지로 이동했습니다.
✅ 51개의 게시글을 발견했습니다.

🎉 크롤링이 성공적으로 완료되었습니다!
총 506개의 게시글을 수집했습니다.

--- 수집 결과 (일부) ---
{'페이지': 1, '번호': '-', '제목': '생활고나 번아웃 고백이 와닿지 않는 스타는?', '글쓴이': None}
{'페이지': 1, '번호': '572295', '제목': '특갤 용어 모음집', '글쓴이': 'ㅇㅇ'}
{'페이지': 1, '번호': '535136', '제목': 'AGI 분류 체계', '글쓴이': 'ㅇㅇ'}
{'페이지': 1, '번호': '819971', '제목': 'AI가 연구자들 사이에서 어떻게 쓰일까?', '글쓴이': '행복한천국으로'}
{'페이지': 1, '번호': '819952', '제목': 'Ai 바비인형 나온다ㄷ', '글쓴이': 'chatgpt4o'}
...
{'페이지': 10, '번호': '776490', '제목': 'GPT-5 8월 초 출시', '글쓴이': 'ㅇㅇ'}
{'페이지': 10, '번호': '776441', '제목': '렙틸리언이 움직이기 시작했다', '글쓴이': 'ㅇㅇ'}
{'페이지': 10, '번호': '776434', '제목': "AI, '약물화 불가능' 단백질